### DSML investigation

You are part of the Suisse Impossible Mission Force, or SIMF for short. You need to uncover a rogue agent that is trying to steal sensitive information.

Your mission, should you choose to accept it, is to find that agent before stealing any classified information. Good luck!

# Assignement part four

#### Identifying the suspects' credit score
We received informations that the rogue agent has a *good* credit score.

Our spies at SIMF have managed to collect financial information relating to our suspects as well as a training dataset.

Create a Neural Network over the training dataset `df` to identify which of the suspects have a *standard* `Credit_Mix`.


## Getting to know our data

* `Age`: a user's age

* `Occupation`: a user's employment field

* `Annual_Income`: a user's annual income

* `Monthly_Inh_Salary`: the calculated salary received by a given user on a monthly basis

* `Num_Bank_Accounts`: the number of bank accounts possessed by a given user

* `Num_Credit_Cards`: the number of credit cards a given user possesses

* `Interest_Rate`: The interest rate on those cards (if multiple then it's the average)

* `Num_of_Loans`: The number of loans of each user

* `Delay_from_due_date`: payment tardiness of user

* `Num_of_Delayed_Payment`: the count of delayed payments

* `Changed_Credit_Limit`: NaN

* `Num_Credit_Inquiries`: NaN

* `Credit_Mix`: The user's credit score

* `Outsting_Debt`: Outstanding debt

* `Credit_Utilization_Ratio`: the percentage of borrowed money over borrowing allowance

* `Payment_of_Min_Amount`: does the user usually pay the minimal amount (categorical)

* `Total_EMI_per_month`: Monthly repayments to be made

* `Amount_invested_monthly`: The amount put in an investment fund by the user on a monthly basis

* `Payment_Behaviour`: the user's payment behavior (categorical)

* `Monthly_Balance`: The user's end of the month balance

* `AutoLoan`: If the user has an active loan for their vehicle

* `Credit-BuilderLoan`: If the user has a loan to increase their credit score

* `DebtConsolidationLoan`, `HomeEquityLoan`, `MortgageLoan`, `NotSpecified`, `PaydayLoan`, `PersonalLoan`, `StudentLoan`: different types of loans (categorical features)


In [ ]:
# Import required packages

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/master/Assignements/Part%204/data/train_classification.csv", index_col='Unnamed: 0').dropna()
suspects = pd.read_csv("https://raw.githubusercontent.com/michalis0/DataScience_and_MachineLearning/master/Assignements/Part%204/data/suspects.csv", index_col='Unnamed: 0').dropna()
suspects.rename(columns={"Payment_Behaviour": "le_Payment_Behaviour", "Payment_of_Min_Amount": "le_Payment_of_Min_Amount"}, inplace=True)

In [ ]:
#df = df.reset_index(drop=True, inplace=True)
#suspects = suspects.reset_index(drop=True, inplace=True)


display(df.head())
print(df.shape)
display(suspects.head())
print(suspects.shape)

#print(df.columns)
#print(suspects.columns)

#print(df.index)
#print(suspects.index)

#display(df.info())
#display(df.describe())
#display(df.dtypes)
#display(df.value_counts())

#display(suspects.info())
#display(suspects.describe())
#display(suspects.dtypes)
#display(suspects.value_counts())

,Age,Occupation,Annual_Income,Monthly_Inh_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,...,Monthly_Balance,AutoLoan,Credit-BuilderLoan,DebtConsolidationLoan,HomeEquityLoan,MortgageLoan,NotSpecified,PaydayLoan,PersonalLoan,StudentLoan
0,23,Scientist,19114.12,1824.843333,3,4,3,4,3,7,...,186.266702,1,1,0,1,0,0,0,1,0
1,24,Scientist,19114.12,1824.843333,3,4,3,4,3,9,...,361.444004,1,1,0,1,0,0,0,1,0
3,24,Scientist,19114.12,4182.004291,3,4,3,4,4,5,...,343.826873,1,1,0,1,0,0,0,1,0
5,28,Teacher,34847.84,3037.986667,2,4,6,1,3,3,...,303.355083,0,1,0,0,0,0,0,0,0
8,35,Engineer,143162.64,4182.004291,1,5,8,3,8,1942,...,854.226027,2,0,0,0,0,1,0,0,0


(29223, 29)


,Age,Annual_Income,Monthly_Inh_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,...,Occupation_Journalist,Occupation_Lawyer,Occupation_Manager,Occupation_Mechanic,Occupation_MediaManager,Occupation_Musician,Occupation_Scientist,Occupation_Teacher,Occupation_Writer,userID
0,23.0,19114.12,1824.843333,3.0,4.0,3.0,4.0,3.0,7.0,11.27,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,317991
1,24.0,19114.12,1824.843333,3.0,4.0,3.0,4.0,3.0,9.0,13.27,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,241892
3,24.0,19114.12,4182.004291,3.0,4.0,3.0,4.0,4.0,5.0,11.27,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,303376
5,28.0,34847.84,3037.986667,2.0,4.0,6.0,1.0,3.0,3.0,5.42,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,761992
8,35.0,143162.64,4182.004291,1.0,5.0,8.0,3.0,8.0,1942.0,7.10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,373318


(714, 43)


In [ ]:
df["Credit_Mix"].unique()

array(['Good', 'Standard', 'Bad'], dtype=object)

# 1. Preparing the data
## 1.1 Data cleaning
Consider the dataset loaded into the DataFrame `df` which is *train_classification.csv*. We aim to preprocess this data for model training.

Begin by encoding the categorical variables:
- Apply One-Hot Encoding to `Occupation`
- Apply Label Encoding to `Payment_of_Min_Amount` and `Payment_Behaviour`

*Note: To clearly distinguish your newly encoded columns, especially for label encoding, consider renaming them with a prefix. Please, use `le_Payment_of_Min_Amount` and `le_Payment_Behaviour` for the label-encoded new columns.*

In [ ]:
# Fit and transform with OneHotEncoder for the "Occupation" column
ohe_fit_Occupation = OneHotEncoder().fit(df[["Occupation"]])
ohe_Occupation = ohe_fit_Occupation.transform(df[["Occupation"]]).toarray()

# Create DataFrame from the one-hot encoded array and set the index to match df's index
ohe_Occupation = pd.DataFrame(
    ohe_Occupation,
    columns=ohe_fit_Occupation.get_feature_names_out(),
    index=df.index  # Set the index to match the original df's index !!!!!! très important!!!!
)

# Fit and transform with LabelEncoder for the "Payment_of_Min_Amount" column
le_fit_Payment_of_Min_Amount = LabelEncoder().fit(df["Payment_of_Min_Amount"])
le_Payment_of_Min_Amount = pd.DataFrame(
    le_fit_Payment_of_Min_Amount.transform(df["Payment_of_Min_Amount"]),
    columns=["le_Payment_of_Min_Amount"],
    index=df.index  # Set the index to match the original df's index
)

# Fit and transform with LabelEncoder for the "Payment_Behaviour" column
le_fit_Payment_Behaviour = LabelEncoder().fit(df["Payment_Behaviour"])
le_Payment_Behaviour = pd.DataFrame(
    le_fit_Payment_Behaviour.transform(df["Payment_Behaviour"]),
    columns=["le_Payment_Behaviour"],
    index=df.index  # Set the index to match the original df's index
)

# Concatenate the results side by side, aligning by index
combined_encoded = pd.concat([ohe_Occupation, le_Payment_of_Min_Amount, le_Payment_Behaviour], axis=1)

# Display the first 10 rows to verify the results
display(combined_encoded.head(10))
print(combined_encoded.shape)

,Occupation_Accountant,Occupation_Architect,Occupation_Developer,Occupation_Doctor,Occupation_Engineer,Occupation_Entrepreneur,Occupation_Journalist,Occupation_Lawyer,Occupation_Manager,Occupation_Mechanic,Occupation_MediaManager,Occupation_Musician,Occupation_Scientist,Occupation_Teacher,Occupation_Writer,le_Payment_of_Min_Amount,le_Payment_Behaviour
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,6
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,2
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,2
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,4
8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,5
9,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,4
10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,5
12,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2
14,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,5
15,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6


(29223, 17)


 After encoding, integrate the newly encoded columns back into a new DataFrame named `df_encoded`, and remove the original columns `Occupation`, `Payment_of_Min_Amount`, and `Payment_Behaviour` to avoid redundancy.

In [ ]:
df_encoded = pd.concat([df, ohe_Occupation, le_Payment_of_Min_Amount, le_Payment_Behaviour], axis=1)
df_encoded.drop(columns=["Occupation", "Payment_of_Min_Amount", "Payment_Behaviour"], inplace=True)

Finally, display the first few rows of `df_encoded` to verify that the encodings are correctly implemented. This prepared DataFrame will be used for subsequent model training.

In [ ]:
display(df_encoded.head())
print(df_encoded.shape)

,Age,Annual_Income,Monthly_Inh_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,...,Occupation_Lawyer,Occupation_Manager,Occupation_Mechanic,Occupation_MediaManager,Occupation_Musician,Occupation_Scientist,Occupation_Teacher,Occupation_Writer,le_Payment_of_Min_Amount,le_Payment_Behaviour
0,23,19114.12,1824.843333,3,4,3,4,3,7,11.27,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,6
1,24,19114.12,1824.843333,3,4,3,4,3,9,13.27,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,2
3,24,19114.12,4182.004291,3,4,3,4,4,5,11.27,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,2
5,28,34847.84,3037.986667,2,4,6,1,3,3,5.42,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,4
8,35,143162.64,4182.004291,1,5,8,3,8,1942,7.10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,5


(29223, 43)


## 1.2 Dataset splitting and rescaling

To effectively train and validate our model, it is crucial to properly prepare and partition the data. Follow these steps to preprocess and split the dataframe df_encoded into training and test subsets, ensuring that our model can generalize well to new data:

- Set `X` as all columns except `Credit_Mix` and `y` as the dependent feature `Credit_Mix`.
- Apply manually Label Encoding to `y` using the function `.map`and encoding with the following : Good=2, Standard=1, Bad=0.
- Use a `random_state` of 42 to split `X` and the encoded `y` into training (80%) and test sets (20%).
- Normalize `X` using `MinMaxScaler()`.

In [ ]:
df_encoded["Credit_Mix"] = df_encoded["Credit_Mix"].map({"Good": 2, "Standard": 1, "Bad": 0}) #should I drop nan values???
#df_encoded.dropna(inplace=True)#, how="all") #yes! nwo there are no nan value, since the index for ohe match the index of df.
#print(df_encoded["Credit_Mix"].unique()) show [2 1 0] (already test there are only Good, standard, and bad on the code above)
X = df_encoded.drop(columns=["Credit_Mix"])
y = df_encoded["Credit_Mix"] #!!!! need to be but [[""]] so it give the 1dimesnional later!! False, need to be like that 1dimensional


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### 1.2.2 Final touches
Convert the features to torch tensors of type `torch.float` and the labels (dependent variables) to torch tensors of type `torch.long`.

In [ ]:
X_train = torch.tensor(X_train, dtype=torch.float)
y_train = torch.tensor(y_train.values, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float)
y_test = torch.tensor(y_test.values, dtype=torch.long)
print(X_train.size(), y_train.size())

torch.Size([23378, 42]) torch.Size([23378])


# 2 Model preparation:

## 2.1 Define a Neural network model and instantiate it.
Define your neural network model as a class in PyTorch, extending from nn.Module. In the `__init__` method, initialize a linear layer using `nn.Linear()` with specified input and output sizes, and set up `nn.ReLU()` for activation. Implement the forward method to describe how data passes through this layer during the network's forward computation.

Set the following parameters:
* `hidden layer` : 1
* `activation function` : ReLU

In [ ]:
class Net(nn.Module):
    def __init__(self, D_in, H1, D_out):
        super(Net, self).__init__()

        self.linear1 = nn.Linear(D_in, H1)        # Linear transformation for hidden layer
        self.linear2 = nn.Linear(H1, D_out)       # Linear transformation for output layer
        self.activation = nn.ReLU()               # Activation function for hidden layer

    def forward(self, x):
        y_pred = self.activation(self.linear1(x))   # Hidden layer: linear transformation + ReLU
        y_pred = self.linear2(y_pred)               # Output layer: linear transformation
        return y_pred

Set `D_in` to the number of features in `X_train` and `D_out` to the number of target variables in `y_train`, then print these dimensions to verify their values.

In [ ]:
D_in, D_out = X_train.shape[1], len(y_train.unique())#y_train.shape[1]

print(D_in, D_out)

42 3



Initialize the `Net` model with the specified input size `D_in`, 150 hidden units, and output size `D_out`.

In [ ]:
# Model with 150 neurons
model1 = Net(D_in, 150, D_out)

Let's calculate now how many parameters we have in the model.

In [ ]:
# calculate how many parameters are in the model
pytorch_total_params = sum(p.numel() for p in model1.parameters() if p.requires_grad)
print(pytorch_total_params)

6903


**Q1. How many parameters does your model have ?** 6903

*Note: Enter an integer (e.g. 355)*

## 2.2 Finding the best model

Determine the optimal hyper-parameters for your model from the options listed below:

* `criterion` : [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)
* `optimizer` : [Stochastic Gradient Descent (SGD)](https://en.wikipedia.org/wiki/Stochastic_gradient_descent)
* `Epochs`: Test with **150**, **250**, **500**, and **1000** epochs.
* `Learning Rate`: Experiment with learning rates of **0.00005**, **0.001**, **1**, and **10**.

**Evaluation**: Assess your model's performance by measuring its accuracy on the test set.

*Note: Run the code `torch.manual_seed(42)` to ensure consistency across all experiments.*

In [ ]:
torch.manual_seed(42)   # Set the seed for reproducibility

### 2.2.1 Automatically Tuning Hyperparameters

In this section, you will automate the process of testing different hyperparameter combinations using a loop, as demonstrated in the lab.

Begin by defining the ranges for the hyperparameters you want to explore:
- **Epochs**: Test with **150**, **250**, **500**, and **1000** epochs.
- **Learning Rate**: Experiment with learning rates of **0.00005**, **0.001**, **1**, and **10**.


In [ ]:
epochs = [150, 250, 500, 1000]
learning_rates = [0.00005, 0.001, 1, 10]

Next, create a loop that iterates over the list of learning rates, with an inner loop iterating over the list of number of epochs. Inside the inner loop, initialize (define) the model, and also define the optimizer and the loss function (criterion). Then train the model as demonstrated in the lab, and after the training evaluate its performance to obtain the test accuracy for each model. Ensure that you also display the test loss, as you will need it to answer the upcoming questions.

Here’s a reminder of the criterion and optimizer you should use:
- **Criterion**: [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)
- **Optimizer**: [Stochastic Gradient Descent (SGD)](https://en.wikipedia.org/wiki/Stochastic_gradient_descent)

**Hint**: Make sure to define the criterion and optimizer at every iteration (inside the inner loop), otherwise the model might retain previous training states and produce biased results.

(Optional) You can enhance the code by storing the best model along with its best_accuracy and best_params. This way, you'll have an automatic evaluation at the end to identify the best-performing model.



In [ ]:
from tqdm import tqdm

# Assuming model1 is a class that extends nn.Module, with a defined structure.
# Initialize the model parameters, loss function, and optimizer for the loop
best_accuracy = 0  # Variable to store the best accuracy
best_model = None  # Variable to store the best model parameters
best_params = {}   # Dictionary to store the best hyperparameters

# Initialize a list to store the results
results = []

# Loop through learning rates and epochs
for learning_rate in learning_rates:
    for epoch in epochs:
        # Initialize the model for each combination of learning rate and epoch
        model1 = Net(D_in, 150, D_out)  # Replace D_in, H1, D_out with your actual values
        model1.train()  # Set the model to training mode

        # Define the criterion (loss function) for classification
        criterion = nn.CrossEntropyLoss()

        # Define the SGD optimizer with the current learning rate
        optimizer = optim.SGD(model1.parameters(), lr=learning_rate)

        # Training loop for the specified number of epochs
        for t in tqdm(range(epoch), desc=f'Training LR: {learning_rate}, Epochs: {epoch}', leave=False):
            model1.train()  # Ensure model is in training mode

            # Forward pass: compute predictions on the training set
            y_pred = model1(X_train)  # Pass training data through the model

            # Compute the loss using the criterion
            loss = criterion(y_pred, y_train.squeeze())  # Use squeeze to convert [N, 1] to [N]

            if torch.isnan(loss):
                print("Loss is NaN, stopping training.")
                break  # Stop training if loss is NaN

            # Reset gradients before the backward pass
            optimizer.zero_grad()

            # Backward pass: compute gradients
            loss.backward()

            # Update model parameters using the optimizer
            optimizer.step()

        # After training, evaluate the model on the test set
        model1.eval()  # Set the model to evaluation mode
        with torch.no_grad():  # Disable gradient calculations for evaluation
            output_test = model1(X_test)  # Get predictions for the test set
            test_loss = criterion(output_test, y_test.squeeze())  # Calculate loss on the test set

            # Get predicted classes
            _, predicted_classes = torch.max(output_test, 1)  # Class with the highest score
            test_accuracy = (predicted_classes == y_test.squeeze()).float().mean()  # Compute accuracy

            # Store the results in the list
            results.append({
                'learning_rate': learning_rate,
                'epochs': epoch,
                'test_loss': test_loss.item(),
                'test_accuracy': test_accuracy.item()
            })

            # Print the results
            print(f"Learning Rate: {learning_rate}, Epochs: {epoch}, Test Loss: {test_loss.item()}, Test Accuracy: {test_accuracy.item()}")

            # Check if the current model is the best one
            if test_accuracy > best_accuracy:
                best_accuracy = test_accuracy
                best_model = model1.state_dict()  # Store the state of the best model
                best_params = {'learning_rate': learning_rate, 'epochs': epoch}  # Store the best parameters

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Round the 'test_loss' and 'test_accuracy' columns to 2 decimal places
results_df['test_loss'] = results_df['test_loss'].round(2)
results_df['test_accuracy'] = results_df['test_accuracy'].round(2)

# At the end, you can display the best model and its parameters
print(f"Best Test Accuracy: {best_accuracy.item()}")
print(f"Best Parameters: {best_params}")

# Display the DataFrame
display(results_df)


Learning Rate: 5e-05, Epochs: 150, Test Loss: 1.0904748439788818, Test Accuracy: 0.30966639518737793


Learning Rate: 5e-05, Epochs: 250, Test Loss: 1.1042519807815552, Test Accuracy: 0.2865697145462036


Learning Rate: 5e-05, Epochs: 500, Test Loss: 1.0968035459518433, Test Accuracy: 0.3334473967552185


Learning Rate: 5e-05, Epochs: 1000, Test Loss: 1.0884373188018799, Test Accuracy: 0.43798118829727173


Learning Rate: 0.001, Epochs: 150, Test Loss: 1.0929971933364868, Test Accuracy: 0.3931565582752228


...

Learning Rate: 10, Epochs: 150, Test Loss: 2.5533859729766846, Test Accuracy: 0.4545765519142151


Learning Rate: 10, Epochs: 250, Test Loss: 2.228132963180542, Test Accuracy: 0.4545765519142151


Learning Rate: 10, Epochs: 500, Test Loss: 2.204747200012207, Test Accuracy: 0.4545765519142151


Learning Rate: 10, Epochs: 1000, Test Loss: 2.203885316848755, Test Accuracy: 0.4545765519142151
Best Test Accuracy: 0.8477330803871155
Best Parameters: {'learning_rate': 1, 'epochs': 1000}


,learning_rate,epochs,test_loss,test_accuracy
0,0.00005,150,1.09,0.31
1,0.00005,250,1.10,0.29
2,0.00005,500,1.10,0.33
3,0.00005,1000,1.09,0.44
4,0.00100,150,1.09,0.39
5,0.00100,250,1.06,0.45
6,0.00100,500,1.07,0.45
7,0.00100,1000,1.05,0.45
8,1.00000,150,0.40,0.82
9,1.00000,250,0.38,0.83


In [ ]:
# Q2: What is the test accuracy when we train the model with a learning rate of 0.001 and for 150 epochs?
q2_accuracy = results_df[(results_df['learning_rate'] == 0.001) & (results_df['epochs'] == 150)]['test_accuracy'].values

if q2_accuracy.size > 0:
    q2_accuracy_rounded = round(q2_accuracy[0], 2)  # Round to 2 decimal points
    print(f"Q2 Test Accuracy: {q2_accuracy_rounded}")  # Output: e.g. 0.39
else:
    print("No data found for Q2.")


# Q3: When using 1000 epochs, which learning rate results in the highest test accuracy?
q3_results = results_df[results_df['epochs'] == 1000]  # Filter for 1000 epochs

if not q3_results.empty:
    q3_best_row = q3_results.loc[q3_results['test_accuracy'].idxmax()]  # Get the row with the highest accuracy
    q3_best_learning_rate = q3_best_row['learning_rate']
    q3_best_accuracy = q3_best_row['test_accuracy']

    print(f"Q3 Best Learning Rate: {q3_best_learning_rate}, Test Accuracy: {q3_best_accuracy}")
else:
    print("No data found for Q3.")

Q2 Test Accuracy: 0.39
Q3 Best Learning Rate: 1.0, Test Accuracy: 0.85


### 2.2.3 Questions

**Q2. What is the test accuracy when we train the model with a learning rate of 0.001 and for 150 epochs? Round your answer to 2 decimal points, e.g., 0.25.** **The answer is 0.39**, i.e. 0.3931565582752228 rounded to two decimal


**Q3. When using 1000 epochs, which learning rate results in the highest test accuracy?** **1**

*Note: Select among the following answers*


**Q4. Is BCELoss a suitable alternative to CrossEntropyLoss for our dataset?** **NO**. We should use BCELoss if Credit_Mix has only two unique value, and CrossEntropyLoss if it has more than 2 values (e.g here there are three 0, 1, 2).

*Hint: Consider the unique values in the Credit_Mix output variable when answering.*

### 3. Predict on the Suspects Dataset

Now it's time to use your trained model to make predictions on the suspects dataset!

Use the following parameters for the model:
- **Hidden layer**: 1 hidden layer with 150 neurons
- **Output layer**: 3 neurons for classification
- **Optimizer**: [Stochastic Gradient Descent (SGD)](https://en.wikipedia.org/wiki/Stochastic_gradient_descent)
- **Criterion**: [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)
- **Iterations**: 1000
- **Learning rate**: 1.0

Ensure consistency by setting the manual seed with `torch.manual_seed(42)` before training.

In [ ]:
# Set the manual seed for consistency
torch.manual_seed(42)

# Define input and output dimensions (same as before)
D_in = X_train.shape[1]  # Number of features in the training data
D_out = 3                 # Output classes (as specified in the prompt)

# Initialize the model
model = Net(D_in, 150, D_out)  # 1 hidden layer with 150 neurons, output layer with 3 neurons

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.SGD(model.parameters(), lr=1.0)  # Stochastic Gradient Descent


Now, train your model on the training dataset just as you did in section 2.2.1, but you don't have to loop over all the hyper parameters. Ensure that you use the correct number of epochs specified earlier.

In [ ]:
# Number of epochs
epochs = 1000

# Training loop
for epoch in range(epochs):
    model.train()  # Set the model to training mode

    # Forward pass: compute predictions
    y_pred = model(X_train)  # Forward pass through the model

    # Compute the loss
    loss = criterion(y_pred, y_train)

    # Reset gradients before the backward pass
    optimizer.zero_grad()

    # Backward pass: compute gradients
    loss.backward()

    # Update model parameters
    optimizer.step()

Before making predictions, confirm that the feature column names in the `suspects` dataset match those expected by the model, particularly ensuring the X features correspond accurately.

In [ ]:
# Inspect the columns in the suspects dataset
suspects_columns = suspects.columns.tolist()
print("Suspects Dataset Columns:")
print(suspects_columns)

# Assuming `model` is your trained model, retrieve the feature names
model_features = X.columns.tolist()  # For models that support this
print("\nModel Feature Names:")
print(model_features)

# Check for any discrepancies
missing_features = [feature for feature in model_features if feature not in suspects_columns]
extra_features = [column for column in suspects_columns if column not in model_features]

if not missing_features and not extra_features:
    print("\nAll feature columns match. Ready for predictions.")
else:
    if missing_features:
        print("\nMissing features in suspects dataset:")
        print(missing_features)
    if extra_features:
        print("\nExtra features in suspects dataset that are not in model:")
        print(extra_features)

Suspects Dataset Columns:
['Age', 'Annual_Income', 'Monthly_Inh_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Outsting_Debt', 'Credit_Utilization_Ratio', 'le_Payment_of_Min_Amount', 'Total_EMI_per_month', 'Amount_invested_monthly', 'le_Payment_Behaviour', 'Monthly_Balance', 'AutoLoan', 'Credit-BuilderLoan', 'DebtConsolidationLoan', 'HomeEquityLoan', 'MortgageLoan', 'NotSpecified', 'PaydayLoan', 'PersonalLoan', 'StudentLoan', 'Occupation_Accountant', 'Occupation_Architect', 'Occupation_Developer', 'Occupation_Doctor', 'Occupation_Engineer', 'Occupation_Entrepreneur', 'Occupation_Journalist', 'Occupation_Lawyer', 'Occupation_Manager', 'Occupation_Mechanic', 'Occupation_MediaManager', 'Occupation_Musician', 'Occupation_Scientist', 'Occupation_Teacher', 'Occupation_Writer', 'userID']

Model Feature Names:
['Age', 'Annual_Income', 'Monthly_Inh_Salary', 'Num_Ba

In [ ]:
#Delete extra features:
suspects_features = suspects.drop(columns=extra_features)

print("Suspects Dataset Columns after Dropping Extra Features:")
print("same columns?", sorted(suspects.columns.tolist()) == sorted(X.columns.tolist()))

Suspects Dataset Columns after Dropping Extra Features:
same columns? False


Then scale your dataset and convert it into a torch tensor of dtype float, similar to the preprocessing done for the training set in section 1.2.

In [ ]:
# Initialize the MinMaxScaler
#scaler = MinMaxScaler()

# Scale the features
#suspects_features = scaler.fit_transform(suspects_features)
suspects_features = suspects_features[X.columns.tolist()] # order to be on the same
suspects_features = scaler.transform(suspects_features)

# Convert the scaled features into a PyTorch tensor
suspects_features_tensor = torch.tensor(suspects_features, dtype=torch.float)

Make predictions using the trained model and assign the predicted credit score to each user. Ensure to do the following encoding is used for the predicted categories:
* `0` corresponds to bad credit score,
* `1` corresponds to standard credit score,
* `2` corresponds to good credit score.

Use the predictions to add a new column `credit_score` in the `suspects` dataframe that maps the predicted numerical values to the respective credit score categories.

In [ ]:
# Make predictions using the trained model
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient calculations for evaluation
    predictions = model(suspects_features_tensor)  # Use the tensor for predictions
    predicted_classes = torch.argmax(predictions, dim=1)  # Get predicted classes

# Map predicted numerical values to respective credit score categories
credit_score_mapping = {0: 'Bad', 1: 'Standard', 2: 'Good'}

# Add predicted classes to the original suspects DataFrame
suspects['credit_score'] = predicted_classes.numpy()  # Convert tensor to numpy array
suspects['credit_score'] = suspects['credit_score'].map(credit_score_mapping)

# Display the updated DataFrame with the new credit_score column
print(suspects[['credit_score']])  # Show only the relevant columns

     credit_score
0            Good
1            Good
3            Good
5            Good
8            Good
...           ...
1231     Standard
1233     Standard
1235     Standard
1236          Bad
1237          Bad

[714 rows x 1 columns]




As mentioned earlier, we believe the suspect had a "good" credit score. Review the predictions made by the model, then display how many suspects were categorized under each credit score, and extract the `userID`s of those with a "standard" credit score.

In [ ]:
# Count how many suspects are categorized under each credit score
credit_score_counts = suspects['credit_score'].value_counts()
print("\nCount of Suspects by Credit Score Category:")
print(credit_score_counts)

# Extract the userIDs of those with a "standard" credit score
standard_credit_users = suspects[suspects['credit_score'] == 'Standard']['userID']  # Assuming 'userID' is the column name
print("\nUser IDs with a 'standard' credit score:")
print(standard_credit_users.tolist())  # Convert to list for better readability


Count of Suspects by Credit Score Category:
credit_score
Good        263
Standard    257
Bad         194
Name: count, dtype: int64

User IDs with a 'standard' credit score:
[660506, 244162, 353527, 531937, 413250, 512604, 334769, 564061, 306495, 647912, 946059, 917315, 504629, 520229, 861915, 308845, 888670, 449514, 867078, 618568, 512128, 160212, 466624, 903438, 777470, 397912, 123562, 939524, 863210, 717780, 863592, 898254, 424603, 599462, 601265, 402663, 551506, 365406, 688784, 829946, 380835, 823355, 456546, 790046, 932281, 571364, 796491, 133154, 430864, 899581, 240613, 126115, 839591, 453712, 259083, 512432, 874150, 484182, 744124, 641947, 781819, 666304, 619595, 103928, 245167, 893389, 953342, 570734, 743377, 510420, 204180, 564884, 937535, 376743, 767527, 225178, 519735, 706286, 934741, 792996, 140991, 641389, 460980, 241404, 325946, 154013, 537543, 418245, 843485, 99489, 231948, 645264, 649131, 509130, 719655, 316652, 726678, 804662, 456436, 378721, 573810, 769196, 804201, 28

In [ ]:
# List of user IDs to check
user_ids_to_check = [167822, 409055, 519735, 761992, 862880]  # Replace these with the actual user IDs you want to check

# Loop through each user ID and check if it's in the suspects DataFrame
for user_id in user_ids_to_check:
    if user_id in suspects['userID'].values:  # Check if user ID exists
        credit_score = suspects.loc[suspects['userID'] == user_id, 'credit_score'].values[0]
        if credit_score == 'Good':
            print(f"User ID {user_id} has a 'Good' credit score.")
        else:
            print(f"User ID {user_id} does NOT have a 'Good' credit score.")
    else:
        print(f"User ID {user_id} is NOT present in the suspects DataFrame.")


User ID 167822 has a 'Good' credit score.
User ID 409055 has a 'Good' credit score.
User ID 519735 does NOT have a 'Good' credit score.
User ID 761992 has a 'Good' credit score.
User ID 862880 is NOT present in the suspects DataFrame.


**Q5.Which of the following suspects have a "good" credit mix according to your model's predictions?**


## Your investigation is progressing effectively, and the list of suspects is narrowing down.

**Don't forget to answer the quiz and submit your code on Moodle before the end of the deadline.**